In [1]:
import pandas as pd
import plotly.express as px
import plotly
import numpy as np

# Done to avoid flooding the screen with warnings for legacy numpy operations
# in pandas methods
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Assumption: localminute column is clean and error free
def index_and_sort(data): 
    merged_df = pd.DataFrame()

    for k,df in data.groupby(["dataid"]): 
        df.sort_values(by=["localminute"], inplace=True)
        df["val_diff"] = df["meter_value"].diff()

        df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df['localminute'], utc=True, infer_datetime_format=True, cache=True)))
        df.drop(columns=["localminute"], inplace=True)
        
        merged_df = merged_df.append(df)

    return merged_df

In [3]:
def remove_spikes(data_df): 
    spikeless_resampled_df = pd.DataFrame()

    for k, df in data_df.groupby("dataid"): 
        spikeless_df = df[~(df['val_diff'].shift(-1) < 0)] 
        spikeless_df['val_diff'] = spikeless_df['meter_value'].diff()

        # Need to do this because some spikes are less "sharp" than 1 timestemp
        for i in range(10):  # by right should be doing UNTIL no more spikes left. Tested to see no more spikes after 10 passes
            spikeless_df = spikeless_df[~(spikeless_df['val_diff'].shift(-1) < 0)] 
            spikeless_df['val_diff'] = spikeless_df['meter_value'].diff()

        spikeless_sample = spikeless_df.resample('1h').mean()
        spikeless_sample["dataid"].fillna(k, inplace=True)
        spikeless_sample["meter_value"] = spikeless_sample["meter_value"].interpolate()
        spikeless_resampled_df = spikeless_resampled_df.append(spikeless_sample)

    return spikeless_resampled_df

In [4]:
data = pd.read_csv("dataport-export_gas_oct2015-mar2016.csv")
merged_df = index_and_sort(data)
clean_df = remove_spikes(merged_df)

In [5]:
# Hyper-Parameter: 
# How many readings do we need to predict well?  
LOOKBACK_PERIOD = 6

In [6]:
# input frame = [LOOKBACK_PERIOD val_diffs] + [ExpectedOutput] + [HourToPredictFor]
# returns: [inputs], [labels] where correspond by position
def make_io_frames(data_df): 
    input_data = []
    for k, df in data_df.groupby('dataid'): 
        diff_series =  df['meter_value'].diff().dropna()

        # The = + [] is giving the model the hour of day it is predicting for (important)  
        list_of_data = [diff_series[i:i+LOOKBACK_PERIOD+1].tolist() + [diff_series[i:i+LOOKBACK_PERIOD+1].index[-1].hour] for i in range(0,df.shape[0],1) if diff_series[i:i+LOOKBACK_PERIOD+1].shape[0] == LOOKBACK_PERIOD+1]

        for item in list_of_data: 
            input_data.append(item)
   
    X = [frame[:LOOKBACK_PERIOD]+[frame[-1]] for frame in input_data]
    Y = [frame[-2] for frame in input_data]

    return X, Y

In [7]:
# Question: Remove 30% of houses at random first and then make input frames? 
X, Y = make_io_frames(clean_df)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [9]:
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

LinearRegression()

In [10]:
predictions = lin_model.predict(X_train)
mean_squared_error(Y_train, predictions)

49.85421428855172

In [11]:
predictions_test = lin_model.predict(X_test)
mean_squared_error(Y_test, predictions_test)

43.81696857990257

In [12]:
# This is the frontend for the model that preprocesses input_data before  

# Assumptions: 
# input_data contain all data points for the past LOOKBACK+1 hours.
# input data has a DateTimeIndex
# input data is sorted 
# hour_to_predict is a pandas_timestamp. 
def predict(input_data, hour_to_predict, model): 

    latest_available_reading = input_data[input_data.index < hour_to_predict].meter_value[-1]

    #print(input_data)
    #print("Predicting for", hour_to_predict)

    meter_readings = input_data['meter_value']
    meter_readings = meter_readings[(meter_readings.index > (hour_to_predict - pd.Timedelta(str(LOOKBACK_PERIOD)+'h'))) & (meter_readings.index < hour_to_predict)]
    meter_readings = meter_readings.resample('1h').mean()

    #print(meter_readings)

    def getMeterReading(hours_prior):
        req_hour = hour_to_predict - pd.Timedelta(str(hours_prior)+'h')
        #print(req_hour)
        if req_hour in meter_readings.index:
            return meter_readings.loc[req_hour]
        return np.nan

    attributes = pd.Series([getMeterReading(i) for i in range(LOOKBACK_PERIOD+1, 0, -1)])
    attributes = attributes.interpolate()
    #print(attributes)

    diffs = attributes.diff().fillna(0)[1:]
    #print(diffs)
    
    prediction = model.predict([diffs.tolist() + [hour_to_predict.hour]])

    return latest_available_reading + prediction[0]

In [13]:
# architecture for inference
# raw-data -> [PreProcessing] -> [ModelFrontend] -> [Model] -> [Predictions]

# if Predictions = 1 hour only and always corrected, use simulate_operation below. 
# if recursive prediction = predict over predictions, use long_term_prediction below. 

In [14]:
# This is how you use the model
predict(merged_df[:50], pd.Timestamp('2015-10-04 17:00:00+00:00'), lin_model)

93488.94885652313

In [15]:
# Given a house, SIMULATE day to day predictions 
# Take last LOOKBACK PERIOD readings and simulating next hour (time t)
# After actual reading of time t is found, repeat for time t+1. 
# until last known data point. 
# NOTE: Inherent correction in prediction graph. 
def simulate_operation(df_for_house, model):

    timestamps_to_predict_for = df_for_house
    timestamps_to_predict_for = timestamps_to_predict_for.resample('1h').last()
    timestamps = (timestamps_to_predict_for.index)[LOOKBACK_PERIOD+1:]

    predictions = []
    predicted_timestamps = []

    for time in timestamps: 
        try: 
            predictions.append(predict(df_for_house, time, model))
            predicted_timestamps.append(time)
        except: 
            print("ERROR PREDICTING ", time)

    return predicted_timestamps, predictions

In [16]:
# Given a house, SIMULATE day to day predictions 
# Take last LOOKBACK PERIOD readings and simulating next hour (time t)
# assume prediction is correct, repeat for time t+1. 
# NOTE: Long term predicting over predictions. 
def long_term_prediction(df_for_house, time_start, num_days_to_predict, model):

    seed_data = df_for_house[df_for_house.index < time_start]
    seed_data.drop(columns=['val_diff', 'dataid'], inplace=True)

    timestamps = pd.date_range(time_start, periods=num_days_to_predict*24, freq='H')

    predictions = []
    predicted_timestamps = []

    for time in timestamps: 

        try:
            prediction = predict(seed_data, time, model)
            predictions.append(prediction)
            seed_data = seed_data.append(pd.DataFrame(data=[prediction], columns=['meter_value'], index=[time]))
            predicted_timestamps.append(time)
            #print(seed_data)
        except: 
            print("ERROR PREDICTING", time)

    return predicted_timestamps, predictions

In [17]:
timestamps, predictions = simulate_operation(merged_df[merged_df['dataid'] == 35][:250], lin_model)
ltp_timestamps, ltp_predictions = long_term_prediction(merged_df[merged_df['dataid'] == 35][:250], pd.Timestamp('2015-10-02 01:00:00+00:00'), 6, lin_model)

In [18]:
# before plotting actual, need to take to utc because plotly doesn't do it for us.
actual = merged_df[merged_df['dataid'] == 35][:250]
imputed_actual = clean_df[clean_df['dataid'] == 35][:250]

fig = px.scatter()
fig.add_scatter(x=timestamps, y=predictions, name='predictions')
fig.add_scatter(x=actual.index, y=actual['meter_value'].tolist(), name='actual')
fig.add_scatter(x=imputed_actual.index, y=imputed_actual['meter_value'].tolist(), name='imputed')
fig.add_scatter(x=ltp_timestamps, y=ltp_predictions, name='long_term_prediction')

fig.show()

In [19]:
# Observations for single house predictions. 
# 1. Model bery pessimistic - thinks jump is higher than it is. 
# 2. Model has imperfections of imputation, since trained in imputeted data.

In [20]:
# Let's see how model does on mean data. 
def mean_readings_for_area(df): 
    mean_data = df.resample('1h').mean() # get mean val_diffs
    mean_data['meter_value'] = mean_data['val_diff'].cumsum()   # simulate meter_readings from mean_val_diffs
    return mean_data

In [21]:
mean_data = mean_readings_for_area(clean_df)

In [22]:
mean_timestamps, mean_predictions = simulate_operation(mean_data[:250], lin_model)
mean_ltp_timestamps, mean_ltp_predictions = long_term_prediction(mean_data[:250], pd.Timestamp('2015-10-02 01:00:00+00:00'), 6, lin_model)

In [23]:
truth = mean_data[:250]
fig = px.scatter()
fig.add_scatter(x=mean_timestamps, y=mean_predictions, name='predictions')
fig.add_scatter(x=truth.index, y=truth['meter_value'].tolist(), name='mean')
fig.add_scatter(x=mean_ltp_timestamps, y=mean_ltp_predictions, name='long_term_prediction')

fig.show()

In [24]:
# Observation: 
# 1. Model does well on "mean" data for the entire area. 
# 2. Can use this model to predict the average gas usage of the entire area over the next hour. 
# 3. Long Term predictions still VERY poor. 

In [25]:
# TODO 
# 1. Draw Architecture Diagram in paper
# 5. Better test-train split

In [26]:
# Q: Maybe if we ask the model to predict multiple hours in front, it'll temper it's expectations? 
# Did not work, see commit 9b5a028

In [30]:
from sklearn.svm import LinearSVR
svr_lin = LinearSVR(verbose=True)

In [32]:
svr_lin.fit(X_train, Y_train)

[LibLinear]

LinearSVR(verbose=True)

In [33]:
mean_squared_error(svr_lin.predict(X_test), Y_test)

50.60021210474107

In [34]:
timestamps, predictions = simulate_operation(merged_df[merged_df['dataid'] == 35][:250], svr_lin)
ltp_timestamps, ltp_predictions = long_term_prediction(merged_df[merged_df['dataid'] == 35][:250], pd.Timestamp('2015-10-02 01:00:00+00:00'), 6, svr_lin)

In [35]:
# before plotting actual, need to take to utc because plotly doesn't do it for us.
actual = merged_df[merged_df['dataid'] == 35][:250]
imputed_actual = clean_df[clean_df['dataid'] == 35][:250]

fig = px.scatter()
fig.add_scatter(x=timestamps, y=predictions, name='predictions')
fig.add_scatter(x=actual.index, y=actual['meter_value'].tolist(), name='actual')
fig.add_scatter(x=imputed_actual.index, y=imputed_actual['meter_value'].tolist(), name='imputed')
fig.add_scatter(x=ltp_timestamps, y=ltp_predictions, name='long_term_prediction')

fig.show()

In [ ]:
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
# svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
#                coef0=1)